#### Load required Liabraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import shap
from scorecardutils.feature_selection import shap_feature_selection,find_correlation_groups,select_best_features_from_corr_groups
#from feature_engine.selection import SmartCorrelatedSelection


In [ ]:
dev_data = pd.read_csv('../data/credit_risk_dataset.csv')

In [ ]:
dev_data.shape

In [ ]:
dev_data.head()

In [ ]:
## Make sure each feature has correct data types --float,int,catgeory
for col in dev_data.select_dtypes(include='object').columns:
    dev_data[col] = dev_data[col].astype('category')

In [ ]:
## Define target and features names
target = 'default'
features = dev_data.drop(columns=[target]).columns.tolist()

In [ ]:
xgb_params = {
    "objective": "binary:logistic",       # For binary classification
    "eval_metric": "auc",                 # auc
    "learning_rate": 0.05,                 # Smaller means more trees; safer
    "max_depth": 6,                       # Controls model complexity
    "subsample": 0.8,                     # Fraction of samples per tree
    "colsample_bytree": 0.8,              # Fraction of features per tree
    "lambda": 1,                          # L2 regularization
    "alpha": 0,                           # L1 regularization
    "n_estimators": 100,                  # Total trees
}

In [ ]:
selected_features,importance_df,_ =shap_feature_selection(train_data=dev_data,feature_names=features,target_name='default',verbose=True,
                                                        test_size=0.3,random_state=42,use_train_for_shap=False,
                                                        model_params=xgb_params)

In [ ]:
_,importance_df,shapDF =shap_feature_selection(train_data=dev_data,feature_names=selected_features,target_name='default',verbose=False,
                                                        split_data=False,random_state=42,
                                                        model_params=xgb_params,create_shap_df=True)

In [ ]:
shapDF.corr()

In [ ]:
correlated_groups = find_correlation_groups(shapDF, corr_threshold=0.8)

In [ ]:
correlated_groups

In [ ]:
_,selected_features_corr = select_best_features_from_corr_groups(correlated_groups, feature_importance_df=importance_df,
                                      feature_importance_col='SHAP_Importance',
                                      feature_name_col='Feature')

In [ ]:
selected_features_corr

In [ ]:
feature_grp_df